In [90]:
import requests, re
from bs4 import BeautifulSoup

In [29]:
path = "https://economictimes.indiatimes.com/lazyloadlistnew.cms?msid=5766568&curpg=3&img=0"
response = requests.get(path)

In [30]:
response

<Response [200]>

In [31]:
content = response.text

In [32]:
content

'<div class="nat_cont" style="display:inline-block;"><!--esi\n        <esi:assign name="subs_Call">1</esi:assign>\n        <esi:assign name="ads_Call">1</esi:assign>\n        \n        <esi:choose>\n    \t\t<esi:when test="$exists($(HTTP_COOKIE{\'ticket\'})) && $exists($(HTTP_COOKIE{\'et_subs\'}))">\n    \t\t    <esi:comment text="When the user is logged in and is subscribed"/>\n    \t\t    <esi:assign name="subs_Call">0</esi:assign>\n                <esi:assign name="ads_Call">0</esi:assign>\n            </esi:when>\n            <esi:when test="$exists($(HTTP_COOKIE{\'ticket\'})) && !$exists($(HTTP_COOKIE{\'et_subs\'}))">\n    \t\t    <esi:comment text="When the user is logged in but unsure if is subscribed or not"/>\n    \t\t    <esi:assign name="subs_Call">1</esi:assign>\n                <esi:assign name="ads_Call">-1</esi:assign>\n            </esi:when>\n            <esi:when test="!$exists($(HTTP_COOKIE{\'ticket\'}))">\n    \t\t    <esi:comment text="When the user is logged out"/

In [33]:
soup = BeautifulSoup(content, 'html.parser')

In [34]:
print(soup.prettify())

<div class="nat_cont" style="display:inline-block;">
 <!--esi
        <esi:assign name="subs_Call">1</esi:assign>
        <esi:assign name="ads_Call">1</esi:assign>
        
        <esi:choose>
    		<esi:when test="$exists($(HTTP_COOKIE{'ticket'})) && $exists($(HTTP_COOKIE{'et_subs'}))">
    		    <esi:comment text="When the user is logged in and is subscribed"/>
    		    <esi:assign name="subs_Call">0</esi:assign>
                <esi:assign name="ads_Call">0</esi:assign>
            </esi:when>
            <esi:when test="$exists($(HTTP_COOKIE{'ticket'})) && !$exists($(HTTP_COOKIE{'et_subs'}))">
    		    <esi:comment text="When the user is logged in but unsure if is subscribed or not"/>
    		    <esi:assign name="subs_Call">1</esi:assign>
                <esi:assign name="ads_Call">-1</esi:assign>
            </esi:when>
            <esi:when test="!$exists($(HTTP_COOKIE{'ticket'}))">
    		    <esi:comment text="When the user is logged out"/>
    		    <esi:assign name="subs_Ca

In [37]:
stories = soup.find_all("div", {"class": "eachStory"})
stories

[<div class="eachStory"><h3><a href="/markets/stocks/earnings/maruti-suzuki-q1-profit-slumps-27-yoy-to-rs-1436-crore-on-18-drop-in-volumes/articleshow/70393331.cms">Maruti Suzuki Q1 profit slumps 27% YoY to Rs 1,436 crore on 18% drop in volumes</a></h3><time class="date-format" data-time="Jul 26, 2019, 01:49 PM IST">Jul 26, 2019, 01:49 PM IST</time><p>Net sales for the quarter fell 14.1 per cent to Rs 18,735.20 crore on a yearly basis.</p></div>,
 <div class="eachStory"><h3><a href="/markets/stocks/earnings/pnb-posts-surprise-rs-1019-crore-profit-in-q1-as-provisions-plunge-65-yoy/articleshow/70393038.cms">PNB posts surprise Rs 1,019 crore profit in Q1 as provisions plunge 65% YoY</a></h3><time class="date-format" data-time="Jul 26, 2019, 01:34 PM IST">Jul 26, 2019, 01:34 PM IST</time><p>Asset quality deteriorated with percentage of GNPAs rising to 16.49% against 15.50% QoQ.</p></div>,
 <div class="eachStory"><h3><a href="/markets/stocks/earnings/airtel-africa-quarterly-profit-more-than

In [116]:
class Story(object):
    date = None
    title = None
    take_away = None
    summary = None
    body = None
    href = None
    
    def __init__(self):
        pass
        
    def fetch_body(self):
        path = "https://economictimes.indiatimes.com" + self.href
        response = requests.get(path)
        soup = BeautifulSoup(response.text, 'html.parser')
        article = soup.find("article")
        self.summary = article.find("h2", {"class": "title2"}).get_text()
        self.body = article.find("div", {"class": "Normal"}).get_text()
        self.body = re.sub(r"[\n\s]+", " ", self.body)
        
    def __repr__(self):
        return "{date} {title}".format(**self.__dict__)

In [107]:
stories_parsed = [] 
for story in stories:
    story_= Story()
    link = story.find("a")
    story_.href = link.get("href")
    story_.title = link.get_text()
    story_.summary = story.find("p").get_text()
    story_.date = story.find("time").get_text()
    stories_parsed.append(story_)
    
stories_parsed

[Jul 26, 2019, 01:49 PM IST Maruti Suzuki Q1 profit slumps 27% YoY to Rs 1,436 crore on 18% drop in volumes,
 Jul 26, 2019, 01:34 PM IST PNB posts surprise Rs 1,019 crore profit in Q1 as provisions plunge 65% YoY,
 Jul 26, 2019, 01:13 PM IST Airtel Africa Q1 net profit up 40% on year,
 Jul 26, 2019, 01:09 PM IST Bajaj Auto’s margins contract in Q1; key takeaways,
 Jul 26, 2019, 12:41 PM IST Bajaj Auto Q1 profit flat at Rs 1,126 crore, beats Street estimates,
 Jul 26, 2019, 09:43 AM IST Alphabet's revenue rebounds; easing doubts on growth and boosting shares,
 Jul 26, 2019, 09:34 AM IST Amazon's push for one-day delivery dents profits, costs up 21%,
 Jul 25, 2019, 09:15 PM IST Mphasis Q1 profit up 2.4% at Rs 265 crore,
 Jul 25, 2019, 09:10 PM IST Biocon Q1 profit surges 72% to Rs 206 crore,
 Jul 25, 2019, 08:26 PM IST Bank of Baroda Q1 profit at Rs 826 crore,
 Jul 25, 2019, 07:46 PM IST Bajaj Finserv Q1: Check out the key highlights,
 Jul 25, 2019, 06:11 PM IST Tata Motors posts wider-t

In [112]:
path = "https://economictimes.indiatimes.com" + stories_parsed[0].href
path 

'https://economictimes.indiatimes.com/markets/stocks/earnings/maruti-suzuki-q1-profit-slumps-27-yoy-to-rs-1436-crore-on-18-drop-in-volumes/articleshow/70393331.cms'

In [113]:
response = requests.get(path)

In [114]:
soup = BeautifulSoup(response.text, 'html.parser')

In [115]:
len(soup.find_all("article"))

1

In [71]:
article = soup.find("article")
article

<article data-articleid="70392741" data-artidate="Jul 26, 2019, 01.09 PM IST" data-ibeat_author="" data-ibeat_channel="ETMarkets.com" data-ibeat_tag="Bajaj Auto Q1,CVs,Q1 earnings,bajaj auto,indian auto,Motorcycle Company"><input id="article_title" type="hidden" value="Bajaj Auto’s margins contract in Q1; key takeaways"/><input id="article_syn" type="hidden" value="Margins took a beating, falling to 16.1 per cent from 18.3 per cent. "/><input id="article_seo" type="hidden" value="markets/stocks/earnings/bajaj-autos-margins-contract-in-q1-key-takeaways"/><h1 class="clearfix title">Bajaj Auto’s margins contract in Q1; key takeaways</h1><h2 class="title2">Margins took a beating, falling to 16.1 per cent from 18.3 per cent. </h2><div class="clearfix publish_info"><div class="publisher flt">ETMarkets.com<span class="msides5">|</span></div><div class="publish_on flt">Updated: Jul 26, 2019, 07.38 PM IST</div></div><div class="clearfix sharingBox shareVert" data-share_msid="70392741"><a class=

In [76]:
summary = article.find("h2", {"class": "title2"}).get_text()
summary 

'Margins took a beating, falling to 16.1 per cent from 18.3 per cent. '

In [81]:
body = article.find("div", {"class": "Normal"}).get_text()
body

'NEW DELHI: Bajaj Auto on Friday said its standalone profit rose by 1 per cent to Rs 1,126 crore in the June quarter compared with Rs 1,115 crore in the same quarter last year. The profit figure came in better than ET NOW poll estimate of Rs 1,060 crore.\n\nMargins took a beating, falling to 16.1 per cent from 18.3 per cent. Turnover for the quarter rose 5 per cent to Rs 8,197 crore from Rs 7,812 crore in the year-ago period.\n\nEbitda dropped 9 per cent to Rs 1,250 crore against Rs 1,367 crore in the corresponding quarter last year. \n\nHere are key takeaways from Bajaj Auto’s June quarter earnings:\n\n\nMotorcycle - Domestic\n\nThe company recorded a growth of 3 per cent over Q1FY19 as against an industry degrowth of 9 per cent.\n\nOverall share in the domestic motorcycle market was at 18.3 per cent against 16.3 per cent in Q1FY19. Retail share in the domestic motorcycle market is nearly 19 per cent.\n\nIn the entry segment, the company sold over 3,00,000 units and enjoys a market sh

In [97]:
body_clean = re.sub(r"[\n\s]+", " ", body)
body_clean

'NEW DELHI: Bajaj Auto on Friday said its standalone profit rose by 1 per cent to Rs 1,126 crore in the June quarter compared with Rs 1,115 crore in the same quarter last year. The profit figure came in better than ET NOW poll estimate of Rs 1,060 crore. Margins took a beating, falling to 16.1 per cent from 18.3 per cent. Turnover for the quarter rose 5 per cent to Rs 8,197 crore from Rs 7,812 crore in the year-ago period. Ebitda dropped 9 per cent to Rs 1,250 crore against Rs 1,367 crore in the corresponding quarter last year. Here are key takeaways from Bajaj Auto’s June quarter earnings: Motorcycle - Domestic The company recorded a growth of 3 per cent over Q1FY19 as against an industry degrowth of 9 per cent. Overall share in the domestic motorcycle market was at 18.3 per cent against 16.3 per cent in Q1FY19. Retail share in the domestic motorcycle market is nearly 19 per cent. In the entry segment, the company sold over 3,00,000 units and enjoys a market share of 30.6 per cent. Mo

In [119]:
class Story(object):
    date = None
    title = None
    take_away = None
    summary = None
    body = None
    href = None
    
    def __init__(self, story):
        link = story.find("a")
        self.href = link.get("href")
        self.title = link.get_text()
        self.summary = story.find("p").get_text()
        self.date = story.find("time").get_text()
        
    def fetch_body(self):
        path = "https://economictimes.indiatimes.com" + self.href
        response = requests.get(path)
        soup = BeautifulSoup(response.text, 'html.parser')
        article = soup.find("article")
        self.summary = article.find("h2", {"class": "title2"}).get_text()
        self.body = article.find("div", {"class": "Normal"}).get_text()
        self.body = re.sub(r"[\n\s]+", " ", self.body)
        
    def __repr__(self):
        return "{date} {title}".format(**self.__dict__)

    
path = "https://economictimes.indiatimes.com/lazyloadlistnew.cms?msid=5766568&curpg=3&img=0"
response = requests.get(path)

soup = BeautifulSoup(content, 'html.parser')
story_divs = soup.find_all("div", {"class": "eachStory"})

stories = [] 
for div in story_divs:
    try:
        stories.append(Story(div))
    except:
        print("Error", div)

In [120]:
stories

[Jul 26, 2019, 01:49 PM IST Maruti Suzuki Q1 profit slumps 27% YoY to Rs 1,436 crore on 18% drop in volumes,
 Jul 26, 2019, 01:34 PM IST PNB posts surprise Rs 1,019 crore profit in Q1 as provisions plunge 65% YoY,
 Jul 26, 2019, 01:13 PM IST Airtel Africa Q1 net profit up 40% on year,
 Jul 26, 2019, 01:09 PM IST Bajaj Auto’s margins contract in Q1; key takeaways,
 Jul 26, 2019, 12:41 PM IST Bajaj Auto Q1 profit flat at Rs 1,126 crore, beats Street estimates,
 Jul 26, 2019, 09:43 AM IST Alphabet's revenue rebounds; easing doubts on growth and boosting shares,
 Jul 26, 2019, 09:34 AM IST Amazon's push for one-day delivery dents profits, costs up 21%,
 Jul 25, 2019, 09:15 PM IST Mphasis Q1 profit up 2.4% at Rs 265 crore,
 Jul 25, 2019, 09:10 PM IST Biocon Q1 profit surges 72% to Rs 206 crore,
 Jul 25, 2019, 08:26 PM IST Bank of Baroda Q1 profit at Rs 826 crore,
 Jul 25, 2019, 07:46 PM IST Bajaj Finserv Q1: Check out the key highlights,
 Jul 25, 2019, 06:11 PM IST Tata Motors posts wider-t

In [122]:
stories[0].fetch_body()

In [123]:
stories[0].body

'NEW DELHI: India’s largest carmaker Maruti Suzuki on Friday reported a 27.3 per cent year-on-year (YoY) drop in net profit at Rs 1,435.50 crore for the June quarter on weak volumes and higher depreciation expenses. Analysts in an ET NOW poll had projected a profit of Rs 1,350 crore. Net sales for the quarter fell 14.1 per cent to Rs 18,735.20 crore on a yearly basis. The company sold a total of 4,02,594 vehicles during the quarter, down 17.9 per cent. Sales in the domestic market stood at 3,74,481 units, down 19.3 per cent. Exports stood at Rs 28,113 units. The company said that higher sales promotion expenses, depreciation and low capacity utilisation weighed on the numbers. Lower advertisement expenses, cost reduction efforts, favourable exchange movement and favourable commodity prices, were helpful. The auto major’s margins for the quarter came in at 10.4 per cent against ET NOW poll estimate of 10.2 per cent. Ebitda stood at Rs 2,047.80 crore. Analysts had projected at an Ebitda 

In [124]:
story = stories[0]

In [142]:
story.__dict__

{'href': '/markets/stocks/earnings/maruti-suzuki-q1-profit-slumps-27-yoy-to-rs-1436-crore-on-18-drop-in-volumes/articleshow/70393331.cms',
 'title': 'Maruti Suzuki Q1 profit slumps 27% YoY to Rs 1,436 crore on 18% drop in volumes',
 'summary': 'Net sales for the quarter fell 14.1 per cent to Rs 18,735.20 crore on a yearly basis.',
 'date': 'Jul 26, 2019, 01:49 PM IST',
 'body': 'NEW DELHI: India’s largest carmaker Maruti Suzuki on Friday reported a 27.3 per cent year-on-year (YoY) drop in net profit at Rs 1,435.50 crore for the June quarter on weak volumes and higher depreciation expenses. Analysts in an ET NOW poll had projected a profit of Rs 1,350 crore. Net sales for the quarter fell 14.1 per cent to Rs 18,735.20 crore on a yearly basis. The company sold a total of 4,02,594 vehicles during the quarter, down 17.9 per cent. Sales in the domestic market stood at 3,74,481 units, down 19.3 per cent. Exports stood at Rs 28,113 units. The company said that higher sales promotion expenses,

In [129]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [138]:
doc = nlp(story.body)
type(doc)

spacy.tokens.doc.Doc

In [141]:
sentences = list(doc.sents)
len(sentences), sentences

(17,
 [NEW DELHI:,
  India’s largest carmaker Maruti Suzuki on Friday reported a 27.3 per cent year-on-year (YoY) drop in net profit at Rs 1,435.50 crore for the June quarter on weak volumes and higher depreciation expenses.,
  Analysts in an ET NOW poll had projected a profit of Rs 1,350 crore.,
  Net sales for the quarter fell 14.1 per cent to Rs 18,735.20 crore on a yearly basis.,
  The company sold a total of 4,02,594 vehicles during the quarter, down 17.9 per cent.,
  Sales in the domestic market stood at 3,74,481 units, down 19.3 per cent.,
  Exports stood at Rs 28,113 units.,
  The company said that higher sales promotion expenses, depreciation and low capacity utilisation weighed on the numbers.,
  Lower advertisement expenses, cost reduction efforts, favourable exchange movement and favourable commodity prices, were helpful.,
  The auto major’s margins for the quarter came in at 10.4 per cent against ET NOW poll estimate of 10.2 per cent.,
  Ebitda stood at Rs 2,047.80 crore.,

In [146]:
import spacy
nlp = spacy.load("en_core_web_sm")

class Story(object):
    date = None
    title = None
    take_away = None
    summary = None
    body = None
    href = None
    
    def __init__(self, story):
        link = story.find("a")
        self.href = link.get("href")
        self.title = link.get_text()
        self.summary = story.find("p").get_text()
        self.date = story.find("time").get_text()
        
    def exract_sentences(self):
        doc = nlp(self.body)
        self.sentences = list(doc.sents) 
        
    def fetch_body(self):
        path = "https://economictimes.indiatimes.com" + self.href
        response = requests.get(path)
        soup = BeautifulSoup(response.text, 'html.parser')
        article = soup.find("article")
        self.summary = article.find("h2", {"class": "title2"}).get_text()
        self.body = article.find("div", {"class": "Normal"}).get_text()
        self.body = re.sub(r"[\n\s]+", " ", self.body)
        
    def __repr__(self):
        return "{date} {title}".format(**self.__dict__)
    
    
path = "https://economictimes.indiatimes.com/lazyloadlistnew.cms?msid=5766568&curpg=3&img=0"
response = requests.get(path)

soup = BeautifulSoup(content, 'html.parser')
story_divs = soup.find_all("div", {"class": "eachStory"})

stories = [] 
for div in story_divs:
    try:
        stories.append(Story(div))
    except:
        print("Error", div)
        
stories[0].fetch_body()
stories[0].exract_sentences()


In [147]:
stories[0].sentences

[NEW DELHI:,
 India’s largest carmaker Maruti Suzuki on Friday reported a 27.3 per cent year-on-year (YoY) drop in net profit at Rs 1,435.50 crore for the June quarter on weak volumes and higher depreciation expenses.,
 Analysts in an ET NOW poll had projected a profit of Rs 1,350 crore.,
 Net sales for the quarter fell 14.1 per cent to Rs 18,735.20 crore on a yearly basis.,
 The company sold a total of 4,02,594 vehicles during the quarter, down 17.9 per cent.,
 Sales in the domestic market stood at 3,74,481 units, down 19.3 per cent.,
 Exports stood at Rs 28,113 units.,
 The company said that higher sales promotion expenses, depreciation and low capacity utilisation weighed on the numbers.,
 Lower advertisement expenses, cost reduction efforts, favourable exchange movement and favourable commodity prices, were helpful.,
 The auto major’s margins for the quarter came in at 10.4 per cent against ET NOW poll estimate of 10.2 per cent.,
 Ebitda stood at Rs 2,047.80 crore.,
 Analysts had p